In [1]:
!pip install panel==10000

ERROR: Ignored the following versions that require a different python version: 1.3.0 Requires-Python >=3.9; 1.3.0b1 Requires-Python >=3.9; 1.3.0b2 Requires-Python >=3.9; 1.3.0rc1 Requires-Python >=3.9; 1.3.0rc2 Requires-Python >=3.9; 1.3.0rc3 Requires-Python >=3.9; 1.3.1 Requires-Python >=3.9; 1.3.1rc1 Requires-Python >=3.9; 1.3.2 Requires-Python >=3.9; 1.3.2a1 Requires-Python >=3.9; 1.3.2rc1 Requires-Python >=3.9; 1.3.2rc2 Requires-Python >=3.9; 1.3.3 Requires-Python >=3.9; 1.3.3rc1 Requires-Python >=3.9; 1.3.4 Requires-Python >=3.9; 1.3.5 Requires-Python >=3.9; 1.3.5a1 Requires-Python >=3.9; 1.3.5rc1 Requires-Python >=3.9; 1.3.5rc2 Requires-Python >=3.9; 1.3.6 Requires-Python >=3.9; 1.3.6rc1 Requires-Python >=3.9; 1.3.7 Requires-Python >=3.9; 1.3.7rc1 Requires-Python >=3.9; 1.3.7rc2 Requires-Python >=3.9; 1.3.7rc3 Requires-Python >=3.9; 1.3.8 Requires-Python >=3.9; 1.3.8rc1 Requires-Python >=3.9; 1.3.8rc2 Requires-Python >=3.9; 1.3.9a1 Requires-Python >=3.9; 1.4.0 Requires-Python >=3

In [1]:
import panel as pn, plotly.graph_objs as go, numpy as np
pn.extension("plotly")

class PlotManager:
    def __init__(self, port=8080, host="http://127.0.0.1"):
        self.port   = port
        self.host   = host
        self.server = None                 # will hold the pn.serve handle
        self.a      = 5                    # mutable shared state

    # ---------- build one dashboard ---------------------------------
    def sine_app(self):
        freq = pn.widgets.FloatSlider(name="Frequency (Hz)", start=0.5, end=10, value=5)

        @pn.depends(freq=freq)
        def plot(freq):
            self.a = freq                  # <-- keep the latest value
            x = np.linspace(0, 4*np.pi, 50)
            fig = go.Figure(go.Scatter(x=x, y=np.sin(freq*x)))
            fig.update_layout(height=300, width=600, margin=dict(l=40, r=20, t=30, b=30))
            return fig

        return pn.Column("### 📉 Interactive Plotly sine wave", freq, plot).servable()

    def sine_appearing_app(self, n_points = 600,
                           freq = None,
                           interval_ms = 50):
        """
        Animated sine-wave plot that reveals one point every *interval_ms*.

        Parameters
        ----------
        n_points     number of samples making up the full curve
        freq         sine frequency -– defaults to self.a (last slider setting)
        interval_ms  time between points, in milliseconds
        """
        # ----- data -------------------------------------------------
        freq = self.a if freq is None else freq
        x_all = np.linspace(0, 4*np.pi, n_points)
        y_all = np.sin(freq * x_all)

        # empty trace – we’ll extend it in-place
        fig = go.Figure(go.Scattergl(x=[], y=[], mode="lines"))
        fig.update_layout(height=300, width=600,
                          margin=dict(l=40, r=20, t=30, b=30),
                         xaxis = dict(range = [0, 12]))
        plot_pane = pn.pane.Plotly(fig)

        # play button
        play_btn = pn.widgets.Button(name="▶ Play", button_type="primary")

        # state shared with the callback
        idx = {'i': 0}
        callback = {'cb': None}            # will hold PeriodicCallback

        def stream():
            """Add the next data point then refresh the pane."""
            i = idx['i']
            if i < n_points:
                # extend existing arrays (tuples → lists for mutability)
                fig.data[0].x += (x_all[i],)
                fig.data[0].y += (y_all[i],)
                plot_pane.param.trigger('object')   # push update to frontend
                idx['i'] += 1
            else:
                # finished – stop the animation and re-enable the button
                callback['cb'].stop()
                play_btn.disabled = False

        def on_play(_):
            """Start a fresh animation run."""
            # reset trace and state
            fig.data[0].x = ()
            fig.data[0].y = ()
            plot_pane.param.trigger('object')
            idx['i'] = 0
            play_btn.disabled = True

            # (re)start periodic callback
            if callback['cb']:             # if an old one exists, stop it
                callback['cb'].stop()
            callback['cb'] = pn.state.add_periodic_callback(
                stream, period=interval_ms)

        play_btn.on_click(on_play)

        return pn.Column("### 🎞️ Animated sine wave (point-by-point)",
                         play_btn, plot_pane).servable()

    # ---------- launch / relaunch the server ------------------------
    def run(self, db):
        if self.server:                    # stop an earlier run if it exists
            self.server.stop()

        self.server = pn.serve(
            db,               # Panel object to serve
            port=self.port,
            address="0.0.0.0",
            threaded=True,                 # non-blocking → kernel stays usable
            show=False,                    # avoid a useless localhost tab
            allow_websocket_origin=["*"]   # or [self.host] for tighter CORS
        )
        print(f"Open → {self.host}:{self.port}  (copy-paste into a browser)")

# ---------- use it --------------------------------------------------
pm = PlotManager(port=8080)

pm.run(pm.sine_appearing_app())
from IPython.display import IFrame
IFrame("http://127.0.0.1:8080/", width = 500, height = 500)

Open → http://127.0.0.1:8080  (copy-paste into a browser)


In [5]:
x_all = np.linspace(0, 4*np.pi, 600)
y_all = np.sin(5 * x_all)

fig = go.Figure(go.Scatter(x=x_all, y=y_all))
fig.update_layout(height=300, width=600,
                          margin=dict(l=40, r=20, t=30, b=30))
fig.show()